# Praca domowa nr 3

In [34]:
import pandas as pd
import numpy as np
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, recall_score, roc_curve, auc, roc_auc_score

In [35]:
#Wczytanie danych
df = pd.read_csv("australia.csv")

### Krótka analiza 

In [36]:
df.head()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,17.9,35.2,0.0,12.0,12.3,48.0,6.0,20.0,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,0,0
1,18.4,28.9,0.0,14.8,13.0,37.0,19.0,19.0,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,0,0
2,19.4,37.6,0.0,10.8,10.6,46.0,30.0,15.0,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,0,0
3,21.9,38.4,0.0,11.4,12.2,31.0,6.0,6.0,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,0,0
4,24.2,41.0,0.0,11.2,8.4,35.0,17.0,13.0,19.0,15.0,1010.7,1007.4,1.0,6.0,33.6,37.6,0,0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56420 entries, 0 to 56419
Data columns (total 18 columns):
MinTemp          56420 non-null float64
MaxTemp          56420 non-null float64
Rainfall         56420 non-null float64
Evaporation      56420 non-null float64
Sunshine         56420 non-null float64
WindGustSpeed    56420 non-null float64
WindSpeed9am     56420 non-null float64
WindSpeed3pm     56420 non-null float64
Humidity9am      56420 non-null float64
Humidity3pm      56420 non-null float64
Pressure9am      56420 non-null float64
Pressure3pm      56420 non-null float64
Cloud9am         56420 non-null float64
Cloud3pm         56420 non-null float64
Temp9am          56420 non-null float64
Temp3pm          56420 non-null float64
RainToday        56420 non-null int64
RainTomorrow     56420 non-null int64
dtypes: float64(16), int64(2)
memory usage: 7.7 MB


In [38]:
df.describe()

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
count,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000,56420.000000
mean,13.464770,24.219206,2.130397,5.503135,7.735626,40.877366,15.667228,19.786778,65.874123,49.601985,1017.239505,1014.795580,4.241705,4.326515,18.204961,22.710333,0.220879,0.220259
std,6.416689,6.970676,7.014822,3.696282,3.758153,13.335232,8.317005,8.510180,18.513289,20.197040,6.909357,6.870892,2.797162,2.647251,6.567991,6.836543,0.414843,0.414425
min,-6.700000,4.100000,0.000000,0.000000,0.000000,9.000000,2.000000,2.000000,0.000000,0.000000,980.500000,977.100000,0.000000,0.000000,-0.700000,3.700000,0.000000,0.000000
25%,8.600000,18.700000,0.000000,2.800000,5.000000,31.000000,9.000000,13.000000,55.000000,35.000000,1012.700000,1010.100000,1.000000,2.000000,13.100000,17.400000,0.000000,0.000000
50%,13.200000,23.900000,0.000000,5.000000,8.600000,39.000000,15.000000,19.000000,67.000000,50.000000,1017.200000,1014.700000,5.000000,5.000000,17.800000,22.400000,0.000000,0.000000
75%,18.400000,29.700000,0.600000,7.400000,10.700000,48.000000,20.000000,26.000000,79.000000,63.000000,1021.800000,1019.400000,7.000000,7.000000,23.300000,27.900000,0.000000,0.000000
max,31.400000,48.100000,206.200000,81.200000,14.500000,124.000000,67.000000,76.000000,100.000000,100.000000,1040.400000,1038.900000,8.000000,9.000000,39.400000,46.100000,1.000000,1.000000


In [39]:
df["RainTomorrow"].value_counts()

0    43993
1    12427
Name: RainTomorrow, dtype: int64

## Przygotowanie zbiorów

In [40]:
#Features
X = df.drop("RainTomorrow", axis=1)
#Target variable
y = df["RainTomorrow"]

#Podział na train-test: target variable jest nieco niezbalansowana,
#ponieważ nigdy nie używałem podziału z zachowaniem proporcji (wykorzystanie parametru stratify)
#to postanowiłem go przetestować, chociaż charakterystyka zbioru nie wymaga tego koniecznie
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=123)

#Z krótkiej analizy wynikło, że zmienne mają znacząco różne zbiory wartości,
#dlatego przeprowadzę skalowanie do przedziału [0,1] zalecane przed użyciem algorytmu SVM
scaler1 = MinMaxScaler()
scaler2 = MinMaxScaler()
X_train = scaler1.fit_transform(X_train)
X_test = scaler2.fit_transform(X_test)

In [48]:
#Funkcja do testowania modeli
def test(X_train, X_test, y_train, y_test, algorytm, proba=True):
    #Trenowanie i predykcja z pomiarem czasu
    t0 = time()
    algorytm.fit(X_train, y_train)
    predict = algorytm.predict(X_test)
    t = round(time()-t0, 3)
    if(proba):
        predict_proba = algorytm.predict_proba(X_test)[:,1]
    #Przypadek dla SVM, gdzie wyliczenie auc wymaga dodatkowej pracy
    else:
        score = algorytm.decision_function(X_test)
        fp_rate, tp_rate, _ = roc_curve(y_test, score)
    
    #Ocena
    acc = accuracy_score(y_test, predict)
    if(proba):
        aucs = roc_auc_score(y_test, predict_proba)
    else:
        aucs = auc(fp_rate, tp_rate)
    rec = recall_score(y_test, predict)
    
    return([acc, rec, aucs, t])

## Algorytm 1: regresja logistyczna
Podstawowy algorytm dla problemu klasyfikacji.\
Wybrane parametry:

- algorytm optymalizacyjny: "liblinear", implementacja algorytmu coordinate descent
- metoda regularyzacji: "l1", funkcją kary za zbytnie dopasowanie
współczynników jest norma pierwsza\
z ich wektora, powinna dobrze współpracować z wybraną metodą optymalizacji przeprowadzając swego rodzaju selekcję zmiennych
- class_weight = 'balanced': rozwiązanie poprawiło wyniki, automatycznie dobiera wagi dla niezbalansowanej zmiennej celu

In [42]:
lr = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced')

lr_sc = test(X_train, X_test, y_train, y_test, lr)

## Algorytm 2: SVM
Zaawansowany algorytm umożliwiający działania w wysoko wymiarowych przestrzeniach poprzez użycie funkcji nazywanych kernelami. Algorytm dzięki temu jest klasyfikatorem nieliniowym.\
Wybrane parametry:

 - kernel: domyślnie używany jest kernel RBF, zbiór danych jest stosunkowo duży dlatego zmienię kernel na wielomianowy stopnia 3, który jest mniej kosztowny obliczeniowo
 - C: parametr regularyzacji, zmniejszę regularyzcję gdyż wielomianowy kernel ma mniejszą tendecję do over-fittingu, default=1, ustawię C=10 (siła regularyzacji jest odwrotnie proporcjonalna do C)
 - class_weight = 'balanced': analogicznie do regresji logistycznej

In [45]:
svm = SVC(kernel='poly', degree=3, C=10, class_weight='balanced')

svm_sc = test(X_train, X_test, y_train, y_test, svm, proba=False)

## Algorytm 3: Gaussian discriminant analysis
Algorytm z grupy generative algorithms. Zakłada, że zmienne pochodzą z wielowymiarowego rozkładu normalnego. Okazuje się jednak, że algorytm z powodzeniem radzi sobie też wtedy gdy założenie to nie jest bliskie spełnienia.\
Wybrane parametry:

- solver: metoda optymalizacji zmieniona z domyślnej na 'lsqr', czyli least squares solutions, która działa tylko dla klasyfikacji
- shrinkage: domyślnie wyłączone, ale dla metody 'lsqr' możemy użyć tego narzędzia, które poprawia estymację macierzy kowariancji, opcja 'auto' automatycznie wyznacza ją z lematu Ledoit-Wolfa

In [46]:
gd = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')

gd_sc = test(X_train, X_test, y_train, y_test, gd)

## Wyniki modeli

In [47]:
results = pd.DataFrame({'log_reg': lr_sc,
                       'svm': svm_sc,
                       'gauss_disc': gd_sc},
                      index=['accuracy', 'recall', 'auc', 'time [s]'])
results

,log_reg,svm,gauss_disc
accuracy,0.790925,0.782559,0.845232
recall,0.822015,0.851625,0.570325
auc,0.882953,0.883349,0.880201
time [s],10.343000,244.862000,0.095000


Wyniki wszystkich modeli są przyzwoite. Algorytm gaussian discriminant ma jednak zbyt niską miarę recall. Miary predykcji dla SVM i regresji logistycznej są bardzo podobne. Prawdopodobnie dopiero szczegółowe dostrojenie hiperparametrów pozwoliło by na zróżnicowanie i poprawienie wyników oraz wybranie lepszego z nich. Na podstawie powyższych wybrał bym regresję logistyczną zważając na czas działania i prostotę algorytmu, jeśli miał bym przedstawić szybkie rozwiązanie. Do dalszego rozwoju wybrał bym SVM, gdyż przez większe znaczenie parametrów ich dostrojenie ma większy potencjał poprawy wyników.

Dodatkową pracę wykonałem w drugim skrypcie.